In [ ]:
# test custom code by Joep
# zie https://github.com/Bluemine-Intelligence/ExaExact/blob/master/exaexact/exact_online.py
# de nodige files staan in de folder exaexact

In [ ]:
# install exaexact  via setup.py
%pip install -e .

In [ ]:
%pip install python-dotenv

In [ ]:
# load env variables from .env file
import os
from dotenv import load_dotenv

load_dotenv()

base_url = os.getenv("BASE_URL")
redirect_uri = os.getenv("REDIRECT_URI")
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [ ]:
# de "oauth dance" is nodig om een token te krijgen
# dit moet eenmalig gebeuren
# daarna kun je de sessie in leven houden met de refresh token

from exaexact.exact_online import ExactOnline
#from getpass import getpass
#client_secret=getpass('Enter your client_secret: ')

exact = ExactOnline.create_session(host=base_url, client_id=client_id, client_secret=client_secret)

In [ ]:
# vraag een token aan
exact.request_token(host=base_url, client_secret=client_secret, redirect_uri=redirect_uri)

In [ ]:
# check dat we de token hebben gekregen
# nu kunnen we de refresh token gebruiken om een nieuwe token te krijgen
# 600 seconden is 10 minuten geldig
exact.oauth.token

In [ ]:
# wie is er nu daadwerkelijk ingelogged?
exact.me._cache

In [ ]:
# een nieuwe token aanvragen om de sessie in leven te houden
exact.request_token(host=base_url, client_secret=client_secret, redirect_uri=redirect_uri)
exact.oauth.token

In [ ]:
division = exact.current_division

In [ ]:
# get today's date
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')
print(today)

In [ ]:
import pandas as pd

# get all active projects
p = exact.get_entity_set('/api/v1/{division}/sync/Project/Projects')
projects = p.get_entities().select('ID,Code,Description,AccountName,StartDate,EndDate,CreatorFullName,Manager').execute()
#projects = p.get_entities().filter(f"EndDate ge DateTime'{today}T00:00:00Z'").execute()
#projects = select_all(p).filter("AccountName eq 'CFIT'").execute()
dfp = pd.DataFrame(_._cache for _ in projects)
dfp.head()


In [ ]:
# active projects
# filter dataframe by EndDate equal to 2000-01-01
dfp = dfp[dfp['EndDate'] == '2000-01-01']
#df = dfp[dfp['EndDate'].isnull()]
dfp.head()

In [ ]:
#dfp['EndDate'] = pd.to_datetime(dfp['EndDate'])
#dfp.dtypes
dfp.describe()

In [ ]:
# get all active employees
e = exact.get_entity_set('/api/v1/{division}/payroll/ActiveEmployments')
employees = e.get_entities().select('Employee,EmployeeFullName,StartDate').execute()
dfe = pd.DataFrame(_._cache for _ in employees)
dfe


In [ ]:
import pandas as pd
from exaexact.exasol_odata import paginate

start_date = '2023-07-01'
timetransactions = exact.get_entity_set('/api/v1/{division}/project/TimeTransactions')
exact_uren = paginate(timetransactions.get_entities().filter(f"Date ge DateTime'{start_date}T00:00:00Z'"))
df = pd.DataFrame(_._cache for _ in exact_uren)

In [ ]:
df.ProjectCode.unique()

In [ ]:
pd.set_option('display.max_columns', None)
df.shape

In [ ]:
# Status of the transaction: 1 = Draft, 2 = Rejected, 10 = Submitted, 11 = Failed on approval, 14 = Processing, 16 = Processing, 19 = Failed while undoing approval, 20 = Final
df[df['HourStatus'] != 10]

df.shape

In [ ]:
# alle input van hugo b257ad8a-bae9-4daa-998a-07fd29d8d7b1	
# driss dcd808b0-862a-4e9b-930e-72ab8000e9a3	
# remy 4e8fe02d-de2c-4ff1-8edb-f32d473bd1bd
#employee_id = 'b257ad8a-bae9-4daa-998a-07fd29d8d7b1'
#dfhk = df[df['Employee'] == employee_id]

#df.iloc[0]['Employee'] == employee_id

In [ ]:
# select columns from dataframe on create date and hourstatus

# 'ID','Date','Employee','HourStatus','Project','ProjectDescription','Activity','ActivityDescription','Notes','Quantity','Unit','UnitDescription','Billable','BillableDescription','BillableAmountFC','BillableAmountDC','CostAmountFC','CostAmountDC','CostPriceFC','CostPriceDC','CostUnit','CostUnitDescription','CostUnitDe

# Status of the transaction: 1 = Draft, 2 = Rejected, 10 = Submitted, 11 = Failed on approval, 14 = Processing, 16 = Processing, 19 = Failed while undoing approval, 20 = Final

dfs = df[['ID','HourStatus','Created', 'Creator', 'CreatorFullName','Employee','Project','Activity','ActivityDescription','Modified', 'Modifier', 'ModifierFullName','ProjectCode', 'Quantity']]

# add two new columns
# event_timestamp and event 
dfs['event_date'] = dfs['Created']
dfs['event'] = 'Draft'
dfs['resource'] = dfs['CreatorFullName']
dfs['org:group'] = dfs['ProjectCode']

dfs.head()

In [ ]:
dfs_draft = dfs[['ID','event_date','event','resource', 'org:group','Quantity']]

In [ ]:
# dictionary with event codes
event_dict = {1: 'Draft', 2: 'Rejected', 10: 'Submitted', 11: 'Failed on approval', 14: 'Processing', 16: 'Processing', 19: 'Failed while undoing approval', 20: 'Final'}

# all other
df2 = dfs[['ID','Modified','HourStatus','ModifierFullName', 'org:group','Quantity']]

# replace event codes with event names
df2['event'] = df2['HourStatus'].map(event_dict)


In [ ]:
# rename columns in df2
df2 = df2.rename(columns={'Modified': 'event_date', 'ModifierFullName': 'resource'})

# drop hourstatus
df2 = df2.drop(['HourStatus'], axis=1)

In [ ]:

df3.head()

In [ ]:
# merge dfs_draft and df2 into new dataframe
df3 = pd.concat([dfs_draft, df2], ignore_index=True)

In [ ]:
df3.event.value_counts()

In [ ]:
# save dataframe df3 to csv
df3.to_csv('exacturen.csv', index=False)

In [ ]:
# load dataframe from csv
df3 = pd.read_csv('exacturen.csv')

In [ ]:
# convert timestamp event_date to datetime format with timezone
df3['event_date'] = pd.to_datetime(df3['event_date'], format="ISO8601")
# show column types of dataframe
df3.dtypes




In [ ]:
# import pm4py processmining library
import pm4py

# load dataframe directly into pm4py
from pm4py.objects.conversion.log import converter as log_converter

# convert dataframe to event log
df3.rename(columns={'event_date': 'time:timestamp', 'ID': 'case:concept:name', 'event': 'concept:name', 'resource': 'org:resource'}, inplace=True)

#event_log = log_converter.apply(df3)
#event_log = pm4py.format_dataframe(df3, case_id='ID', activity_key='event', timestamp_key='event_date', resource='resource')
event_log =df3

process_model = pm4py.discover_bpmn_inductive(event_log)

In [ ]:
pm4py.view_bpmn(process_model)

In [ ]:
pm4py.write_bpmn(process_model, "process_model.bpmn")

In [ ]:
# discover dfg
pm2 = pm4py.discover_dfg(event_log)

In [ ]:
dfg, start_activities, end_activities = pm4py.discover_dfg(event_log)
pm4py.view_dfg(dfg, start_activities, end_activities)

In [ ]:
dfg, start_activities, end_activities = pm4py.discover_performance_dfg(event_log)
pm4py.view_performance_dfg(dfg, start_activities, end_activities)

In [ ]:
%pip install pyvis

In [ ]:
%pip install networkx

In [ ]:
import pyvis

#import networkx as nx

hw_values = pm4py.discover_handover_of_work_network(event_log)
pm4py.view_sna(hw_values)


In [ ]:
pm4py.view_dotted_chart(event_log, format="png", attributes=["concept:name", "org:resource", "org:group"])

In [ ]:
# filter dataframe by month on date column
dfhk['Date'] = pd.to_datetime(dfhk['Date'])

# select only the month november
dfhk[dfhk['Date'].dt.month == 11]

In [ ]:
from exaexact.exasol_odata import paginate, select_all

import pandas as pd

# de financial_sync is een object dat de tabellen van Exact Online beschikbaar maakt
financial_sync = exact.odata('/api/v1/{division}/sync/Financial/')
table = financial_sync.entity_sets.TransactionLines
df = pd.DataFrame(r._cache for r in paginate(select_all(table).filter('Timestamp ge 6599814500L').top(100)))
df

In [ ]:
print(p)

In [ ]:
from exaexact.exasol_odata import paginate, select_all

import pandas as pd

# de financial_sync is een object dat de tabellen van Exact Online beschikbaar maakt
financial_sync = exact.odata('/api/v1/{division}/sync/Financial/')
table = financial_sync.entity_sets.TransactionLines
df = pd.DataFrame(r._cache for r in paginate(select_all(table).filter('Timestamp ge 6599814500L').top(100)))
df

In [ ]:
# /api/v1/{division}/project/Projects

p = exact.odata('/api/v1/{division}/project/Projects')



In [ ]:
df = pd.DataFrame(r._cache for r in paginate(select_all(table).top(100)))
df